In [1]:
import math
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.color_palette("bright")
import matplotlib as mpl
import matplotlib.cm as cm

import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F 
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
print(torch.__version__)

1.8.1


In [3]:
def save_checkpoint(epoch, model, optimizer):
    """
    Save model checkpoint.
    :param epoch: epoch number
    :param model: model
    :param optimizer: optimizer
    """
    state = {'epoch': epoch,
             'model': model,
             'optimizer': optimizer}
    filename = 'checkpoint_resnet18.pth.tar'
    torch.save(state, filename)

class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [4]:
# 数据集读取

epochs = 200
pre_epoch = 0
BATCH_SIZE = 128
LR = 0.01

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = torchvision.datasets.CIFAR10(root='data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)    #核为4，步长为4
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)

In [7]:
checkpoint = 'checkpoint_resnet18.pth.tar'
print_freq = 20

def main():
    """
    Training.
    """
    
    global start_epoch, classes, epoch, checkpoint
    
    # 初始化模型
    
    if checkpoint is None:
        start_epoch = 0
        model = ResNet18()
        optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    else:
        checkpoint = torch.load(checkpoint, map_location='cuda:0')
        start_epoch = checkpoint['epoch'] + 1
        print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
        model = checkpoint['model']
        optimizer = checkpoint['optimizer']
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(start_epoch, epochs):
        
        if epoch == 100:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.1, momentum=0.9, weight_decay=5e-4)
        if epoch == 150:
            optimizer = optim.SGD(model.parameters(), lr=LR*0.01, momentum=0.9, weight_decay=5e-4)
        
        train(train_loader = train_loader,
             model = model,
             criterion=criterion,
             optimizer=optimizer,
             epoch=epoch)
        save_checkpoint(epoch, model, optimizer)
        evaluate(test_loader, model)
        

def train(train_loader, model, criterion, optimizer, epoch):
    
    model = model.train()
    
    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    start = time.time()
    total = 0
    correct = 0
    for i, data in enumerate(train_loader, 0):
        length = len(train_loader)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # forward + backward
        outputs = model(inputs)
        loss = criterion(outputs, labels).to(device)
        loss.backward()
        optimizer.step()
        
        losses.update(loss.item(), inputs.size(0))
        batch_time.update(time.time() - start)
        start = time.time()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1 = open("train_resnet18.txt", "a")
            f1.write('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t\n'.format(epoch, i, len(train_loader),
                                                    batch_time=batch_time,
                                                    data_time=data_time, loss=losses))
            f1.close()
    print(correct/total)
    f2 = open("train_acc_resnet18.txt", "a")
    f2.write('测试分类准确率为：%.3f%%\n' % (100 * correct / total))
    f2.close()
def evaluate(test_loader, model):
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            # 取得分最高的那个类 (outputs.data的索引号)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
            if i % print_freq == 0:
                print('测试分类准确率为：%.3f%%' % (100 * correct / total))
                f3 = open("test_acc_resnet18.txt", "a")
                f3.write('测试分类准确率为：%.3f%%\n' % (100 * correct / total))
                f3.close()
            acc = 100. * correct / total
            best_acc = 85
            if acc > best_acc:
                f3 = open("best_acc_resnet18.txt", "w")
                f3.write("EPOCH=%d,best_acc= %.3f%%" % (epoch + 1, acc))
                f3.close()
                best_acc = acc
if __name__ == '__main__':
    main()


Loaded checkpoint from epoch 176.

Epoch: [176][0/391]	Batch Time 2.742 (2.742)	Data Time 0.000 (0.000)	Loss 0.0010 (0.0010)	
tensor(1.)
Epoch: [176][20/391]	Batch Time 0.175 (0.298)	Data Time 0.000 (0.000)	Loss 0.0026 (0.0020)	
tensor(1.)
Epoch: [176][40/391]	Batch Time 0.174 (0.238)	Data Time 0.000 (0.000)	Loss 0.0031 (0.0018)	
tensor(1.)
Epoch: [176][60/391]	Batch Time 0.175 (0.217)	Data Time 0.000 (0.000)	Loss 0.0008 (0.0017)	
tensor(1.)
Epoch: [176][80/391]	Batch Time 0.175 (0.207)	Data Time 0.000 (0.000)	Loss 0.0006 (0.0017)	
tensor(1.)
Epoch: [176][100/391]	Batch Time 0.187 (0.201)	Data Time 0.000 (0.000)	Loss 0.0011 (0.0016)	
tensor(1.)
Epoch: [176][120/391]	Batch Time 0.186 (0.197)	Data Time 0.000 (0.000)	Loss 0.0017 (0.0016)	
tensor(1.)
Epoch: [176][140/391]	Batch Time 0.175 (0.195)	Data Time 0.000 (0.000)	Loss 0.0012 (0.0017)	
tensor(0.9999)
Epoch: [176][160/391]	Batch Time 0.176 (0.192)	Data Time 0.000 (0.000)	Loss 0.0010 (0.0017)	
tensor(1.0000)
Epoch: [176][180/391]	Batc

Epoch: [179][300/391]	Batch Time 0.350 (0.366)	Data Time 0.000 (0.000)	Loss 0.0010 (0.0016)	
tensor(0.9999)
Epoch: [179][320/391]	Batch Time 0.342 (0.366)	Data Time 0.000 (0.000)	Loss 0.0023 (0.0016)	
tensor(1.0000)
Epoch: [179][340/391]	Batch Time 0.346 (0.365)	Data Time 0.000 (0.000)	Loss 0.0010 (0.0016)	
tensor(1.0000)
Epoch: [179][360/391]	Batch Time 0.172 (0.363)	Data Time 0.000 (0.000)	Loss 0.0009 (0.0016)	
tensor(1.0000)
Epoch: [179][380/391]	Batch Time 0.293 (0.358)	Data Time 0.000 (0.000)	Loss 0.0008 (0.0016)	
tensor(1.0000)
测试分类准确率为：93.000%
测试分类准确率为：94.571%
测试分类准确率为：94.341%
测试分类准确率为：94.426%
测试分类准确率为：94.395%
Epoch: [180][0/391]	Batch Time 2.949 (2.949)	Data Time 0.000 (0.000)	Loss 0.0009 (0.0009)	
tensor(1.)
Epoch: [180][20/391]	Batch Time 0.357 (0.475)	Data Time 0.000 (0.000)	Loss 0.0016 (0.0014)	
tensor(1.)
Epoch: [180][40/391]	Batch Time 0.354 (0.419)	Data Time 0.000 (0.000)	Loss 0.0017 (0.0015)	
tensor(1.)
Epoch: [180][60/391]	Batch Time 0.297 (0.396)	Data Time 0.000 (0.00

Epoch: [183][180/391]	Batch Time 0.330 (0.372)	Data Time 0.000 (0.000)	Loss 0.0006 (0.0016)	
tensor(1.)
Epoch: [183][200/391]	Batch Time 0.328 (0.370)	Data Time 0.000 (0.000)	Loss 0.0007 (0.0016)	
tensor(1.)
Epoch: [183][220/391]	Batch Time 0.346 (0.369)	Data Time 0.000 (0.000)	Loss 0.0012 (0.0016)	
tensor(1.)
Epoch: [183][240/391]	Batch Time 0.362 (0.368)	Data Time 0.000 (0.000)	Loss 0.0016 (0.0015)	
tensor(1.)
Epoch: [183][260/391]	Batch Time 0.346 (0.367)	Data Time 0.000 (0.000)	Loss 0.0020 (0.0016)	
tensor(1.0000)
Epoch: [183][280/391]	Batch Time 0.346 (0.366)	Data Time 0.000 (0.000)	Loss 0.0017 (0.0016)	
tensor(1.0000)
Epoch: [183][300/391]	Batch Time 0.355 (0.366)	Data Time 0.000 (0.000)	Loss 0.0015 (0.0016)	
tensor(1.0000)
Epoch: [183][320/391]	Batch Time 0.373 (0.365)	Data Time 0.000 (0.000)	Loss 0.0015 (0.0016)	
tensor(1.0000)
Epoch: [183][340/391]	Batch Time 0.361 (0.364)	Data Time 0.000 (0.000)	Loss 0.0018 (0.0016)	
tensor(1.0000)
Epoch: [183][360/391]	Batch Time 0.362 (0.36

Epoch: [187][60/391]	Batch Time 0.376 (0.339)	Data Time 0.000 (0.000)	Loss 0.0013 (0.0016)	
tensor(1.)
Epoch: [187][80/391]	Batch Time 0.359 (0.342)	Data Time 0.000 (0.000)	Loss 0.0015 (0.0015)	
tensor(1.)
Epoch: [187][100/391]	Batch Time 0.361 (0.345)	Data Time 0.000 (0.000)	Loss 0.0019 (0.0015)	
tensor(1.)
Epoch: [187][120/391]	Batch Time 0.345 (0.346)	Data Time 0.000 (0.000)	Loss 0.0008 (0.0016)	
tensor(1.)
Epoch: [187][140/391]	Batch Time 0.344 (0.347)	Data Time 0.000 (0.000)	Loss 0.0010 (0.0015)	
tensor(1.)
Epoch: [187][160/391]	Batch Time 0.366 (0.348)	Data Time 0.000 (0.000)	Loss 0.0012 (0.0016)	
tensor(1.0000)
Epoch: [187][180/391]	Batch Time 0.359 (0.349)	Data Time 0.000 (0.000)	Loss 0.0010 (0.0016)	
tensor(1.0000)
Epoch: [187][200/391]	Batch Time 0.362 (0.350)	Data Time 0.000 (0.000)	Loss 0.0011 (0.0016)	
tensor(1.0000)
Epoch: [187][220/391]	Batch Time 0.359 (0.351)	Data Time 0.000 (0.000)	Loss 0.0007 (0.0016)	
tensor(1.0000)
Epoch: [187][240/391]	Batch Time 0.361 (0.352)	Dat

Epoch: [190][360/391]	Batch Time 0.355 (0.345)	Data Time 0.000 (0.000)	Loss 0.0013 (0.0015)	
tensor(1.0000)
Epoch: [190][380/391]	Batch Time 0.375 (0.346)	Data Time 0.000 (0.000)	Loss 0.0044 (0.0015)	
tensor(0.9999)
测试分类准确率为：93.000%
测试分类准确率为：94.619%
测试分类准确率为：94.390%
测试分类准确率为：94.459%
测试分类准确率为：94.444%
Epoch: [191][0/391]	Batch Time 2.990 (2.990)	Data Time 0.000 (0.000)	Loss 0.0029 (0.0029)	
tensor(1.)
Epoch: [191][20/391]	Batch Time 0.172 (0.454)	Data Time 0.000 (0.000)	Loss 0.0052 (0.0018)	
tensor(1.)
Epoch: [191][40/391]	Batch Time 0.299 (0.358)	Data Time 0.000 (0.000)	Loss 0.0008 (0.0017)	
tensor(1.)
Epoch: [191][60/391]	Batch Time 0.187 (0.324)	Data Time 0.000 (0.000)	Loss 0.0019 (0.0017)	
tensor(1.)
Epoch: [191][80/391]	Batch Time 0.359 (0.314)	Data Time 0.000 (0.000)	Loss 0.0018 (0.0016)	
tensor(1.)
Epoch: [191][100/391]	Batch Time 0.399 (0.323)	Data Time 0.000 (0.000)	Loss 0.0017 (0.0016)	
tensor(1.)
Epoch: [191][120/391]	Batch Time 0.361 (0.329)	Data Time 0.000 (0.000)	Loss 0.000

Epoch: [194][260/391]	Batch Time 0.375 (0.343)	Data Time 0.000 (0.000)	Loss 0.0018 (0.0016)	
tensor(1.)
Epoch: [194][280/391]	Batch Time 0.362 (0.344)	Data Time 0.000 (0.000)	Loss 0.0011 (0.0016)	
tensor(1.)
Epoch: [194][300/391]	Batch Time 0.357 (0.344)	Data Time 0.000 (0.000)	Loss 0.0040 (0.0016)	
tensor(1.)
Epoch: [194][320/391]	Batch Time 0.331 (0.345)	Data Time 0.000 (0.000)	Loss 0.0008 (0.0016)	
tensor(1.)
Epoch: [194][340/391]	Batch Time 0.359 (0.346)	Data Time 0.000 (0.000)	Loss 0.0008 (0.0016)	
tensor(1.)
Epoch: [194][360/391]	Batch Time 0.359 (0.346)	Data Time 0.000 (0.000)	Loss 0.0012 (0.0016)	
tensor(1.)
Epoch: [194][380/391]	Batch Time 0.377 (0.347)	Data Time 0.000 (0.000)	Loss 0.0011 (0.0016)	
tensor(1.)
测试分类准确率为：94.000%
测试分类准确率为：94.619%
测试分类准确率为：94.293%
测试分类准确率为：94.377%
测试分类准确率为：94.407%
Epoch: [195][0/391]	Batch Time 2.984 (2.984)	Data Time 0.000 (0.000)	Loss 0.0010 (0.0010)	
tensor(1.)
Epoch: [195][20/391]	Batch Time 0.344 (0.479)	Data Time 0.000 (0.000)	Loss 0.0010 (0.

Epoch: [198][160/391]	Batch Time 0.379 (0.333)	Data Time 0.000 (0.000)	Loss 0.0015 (0.0016)	
tensor(1.0000)
Epoch: [198][180/391]	Batch Time 0.343 (0.336)	Data Time 0.000 (0.000)	Loss 0.0010 (0.0016)	
tensor(1.0000)
Epoch: [198][200/391]	Batch Time 0.376 (0.339)	Data Time 0.000 (0.000)	Loss 0.0008 (0.0016)	
tensor(1.0000)
Epoch: [198][220/391]	Batch Time 0.359 (0.341)	Data Time 0.000 (0.000)	Loss 0.0023 (0.0016)	
tensor(1.0000)
Epoch: [198][240/391]	Batch Time 0.360 (0.341)	Data Time 0.000 (0.000)	Loss 0.0046 (0.0016)	
tensor(1.0000)
Epoch: [198][260/391]	Batch Time 0.346 (0.342)	Data Time 0.000 (0.000)	Loss 0.0011 (0.0016)	
tensor(1.0000)
Epoch: [198][280/391]	Batch Time 0.346 (0.343)	Data Time 0.000 (0.000)	Loss 0.0040 (0.0016)	
tensor(1.0000)
Epoch: [198][300/391]	Batch Time 0.359 (0.343)	Data Time 0.000 (0.000)	Loss 0.0011 (0.0016)	
tensor(1.0000)
Epoch: [198][320/391]	Batch Time 0.346 (0.344)	Data Time 0.000 (0.000)	Loss 0.0020 (0.0016)	
tensor(1.0000)
Epoch: [198][340/391]	Batch 